In [1]:
%load_ext autoreload
%autoreload 2

## Aim

## Imports
Main imports

In [2]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
from pathlib import Path
import sys

Package imports

In [3]:
sys.path.append(r"C:\Users\SachaGobeyn\GitHub\rfactor")
from rfactor.src.flanders.data_processing import (reformat_resolution,
                                                  compute_statistics_inputdata,
                                                  ErosivityData) 

Load completeness per station
Compute completeness for every station for every year
1. Remove each set (year, station) if no data are reported for December or Januari (value: -1)
2. Compute the number of non-NA-values on the non-zero data series (%) (value in [0,1])

__NOTE__: These values are computed based on the original rainfall timeseries of KMI and VMM

In [15]:
fmap_rainfall=  Path(r"C:\Users\SachaGobeyn\Dropbox (Fluves)\FLUVES\PROJECTS\FL_P_2020_DepO_CNWSMod_1\07_results\2019_rfactor\data\v1_rain_data_set_20210318")
fmap_erosivity = Path(r"C:\Users\SachaGobeyn\GitHub\rfactor\rfactor\src\rfactor\results")
path_data_flanders = Path("..","..","src","flanders","data")
txt_files= path_data_flanders / "datafiles_completeness.csv"

In [16]:
data = ErosivityData(fmap_rainfall,fmap_erosivity)
df_files = data.build_data_set(txt_files)
data.load_data(df_files)
df_files.to_csv("overview_files_stations.csv")

100%|██████████████████████████████████████████████████████████████████████████████████| 56/56 [00:19<00:00,  2.82it/s]


PermissionError: [Errno 13] Permission denied: 'overview_files_stations.csv'

## Ukkel

In [11]:
timeseries = [range(1898,2003,1),
              range(2003,2021,1),
              range(1898,2021,1),
              range(1996,2021,1),
              range(1990,2021,1),
              range(1990,2001,1),
              range(2000,2021,1)]
for i in timeseries:
    print(i)
    df_R=data.load_R(["KMI_6447","KMI_FS3"], i)
    print(np.mean(df_R["value"]))

range(1898, 2003)
958.1058095238096
range(2003, 2021)
1277.1105882352942
range(1898, 2021)
1002.5572950819673
range(1996, 2021)
1237.3845833333332
range(1990, 2021)
1226.8233333333333
range(1990, 2001)
1094.6436363636365
range(2000, 2021)
1272.774


In [12]:
len(data.stations)

56

## Flanders and Belgium
**All stations expect Ukkel**

In [13]:
stations_belgium_excl_ukkel = [station for station in data.stations if station not in ["KMI_6447","KMI_FS3"]]
df_R=data.load_R(stations_belgium_excl_ukkel, i)
print(np.mean(df_R["value"]))
len(stations_belgium_excl_ukkel)

1249.8840752351098


54

**All stations except Ukkel and Wallonia**

In [14]:
stations_flanders_excl_ukkel = [station for station in data.stations if station not in  ["KMI_6447","KMI_FS3","KMI_6455","KMI_6459","KMI_6472","KMI_6494","KMI_6484"]]
df_R=data.load_R(stations_flanders_excl_ukkel, i)
print(np.mean(df_R["value"]))
len(stations_flanders_excl_ukkel)

1222.0887452471484


49

## EXTRA

In [ ]:
df_stations_KMI= pd.read_csv(path_data_flanders / "geoinfo_KMI.csv", delimiter=";")
df_stations_VMM = pd.read_csv(path_data_flanders / "geoinfo_VMM.csv", delimiter=",")
df_stations = df_stations_KMI.append(df_stations_VMM)

In [ ]:
df_stations

**Compute statistics**  
Compute general statistics with .describe() functions,  and couple x and y's. Only 

In [ ]:
resmap = "results"
compute_statistics_inputdata(dict_inputdata, resmap, df_stations)

## Compute R-factor

Run model with Matlab in terminal (navigate to *C:\Users\$USERNAME\GitHub\rfactor\rfactor\src\rfactor*, **see README.md**)
    
    matlab -nodisplay -r "main('C:\Users\\$USERNAME\GitHub\rfactor\rfactor\docs\data\example_inputdata')"

Results are saved in *C:\Users\\$USERNAME\GitHub\rfactor\rfactor\src\rfactor*

In [ ]:
fmap = Path(r"C:\Users\SachaGobeyn\GitHub\rfactor\rfactor\src\rfactor\results")

In [ ]:
fmap.exists()

## Figure

In [ ]:
out = dict_df_output_hm["Ukkel_samen"]
plt.figure(figsize=[12,7.5])
col = np.arange(1898,2003,1)
col = [i for i in col if i in dict_df_output_hm["Ukkel_samen"].columns]
plt.plot(col,out[col].iloc[-1],label="KMI_FS3 (1898-2002)")
col = np.arange(2002,2019,1)
col = [i for i in col if i in dict_df_output_hm["Ukkel_samen"].columns]
plt.plot(col,out[col].iloc[-1],label="KMI_6447 (2003-2019)")

# rolling mean
y = out.iloc[-1].rolling(10).mean()
x = y.index
plt.plot(x,y,ls=':',lw=3,label='voortschrijdend gemiddelde (10 jaar)')
y = out.iloc[-1].rolling(20).mean()
x = y.index
plt.plot(x,y,ls='--',lw=3,label='voortschrijdend gemiddelde (20 jaar)')
y = out.iloc[-1].rolling(30).mean()
x = y.index
plt.plot(x,y,ls='-.',lw=3,label='voortschrijdend gemiddelde (30 jaar)')
plt.ylim([0,3500])
plt.xlim([1890,2020])
plt.tick_params(axis='both', which='major', labelsize=14)
plt.xlabel(r'Jaar (-)',fontsize=16)
plt.ylabel(r'Jaarlijkse R [MJ mm ha$^{-1}$ h$^{-1}$ jaar$^{-1}$]',fontsize=16)
plt.legend(prop={'size':16})

Compare results of Ukkel with other average of other stations

In [ ]:
coldf = ["station","rc","offset","afstand","p","r_value","n","dR","meanR"]
out = pd.DataFrame(data = np.zeros([len(dict_df_output_hm)-3,len(coldf)]),columns=coldf)
ind = 0

for i in dict_df_output_hm.keys():
    
    if (i!="Ukkel_samen") & (i!="KMI_FS3") & (i!="KMI_6447"):
        col = dict_df_output_hm["Ukkel_samen"].columns
        col_fill_nan = [j for j in col if j not in dict_df_output_hm[i].columns]
        for j in col_fill_nan:
            dict_df_output_hm[i][j] = np.nan
        y = dict_df_output_hm[i][col].iloc[-1]
        x = dict_df_output_hm["Ukkel_samen"][col].iloc[-1]
        x = x[~np.isnan(y)]        
        y = y[~np.isnan(y)]
        df_i = pd.DataFrame()
        df_i[i] = x.values.flatten()
        df_i["Ukkel (KMI_FS3 & KMI_6447)"] = y.values.flatten()
        df_i = df_i.sort_values("Ukkel (KMI_FS3 & KMI_6447)")
        
        out["dR"].loc[ind] = np.mean(dict_df_output_hm[i].iloc[-1])-np.mean(dict_df_output_hm["Ukkel_samen"].iloc[-1])
        out["meanR"].loc[ind] = np.mean(dict_df_output_hm[i].iloc[-1])
        ind += 1

out.to_csv(Path(resmap) / "stat_allstations_res.csv")